# Import Modules

In [ ]:
# Import modules
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

# Defining global variables

In [ ]:
url_modules = "https://registry.terraform.io/v2/modules"
url_provider = "https://registry.terraform.io/v2/providers"

# Defining functions

In [187]:
# Returns from the given URL the pagination data from JSON
def get_pagination(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data['meta']['pagination']

In [215]:
def get_all_data(url, pages):
    data_modules = []
    for i in range(pages):
        url_page = url + "?page=" + str(i+1)
        response = requests.get(url_page)
        data = json.loads(response.text)
        # convert json data to list
        for item in data['data']:
            data_modules.append(item)

    return data_modules

# Providers

In [195]:
print(get_pagination(url_provider))

{'page-size': 15, 'current-page': 1, 'next-page': 2, 'prev-page': None, 'total-pages': 152, 'total-count': 2267}


# Modules

In [217]:
# Getting the list of all modules of v2 API
#df = pd.DataFrame(data['meta'])
#df.plot(kind='bar')
#plt.show()

In [216]:
# Next iterate over the page of each API call to store all available modules in a list
# Then use this data to filter for provider like aws, google, azurerm to get the count of modules for each provider
# Then plot the count of modules for each provider
print(list(get_all_data(url_modules, get_pagination(url_modules)['total-pages'])))
# FIXME: something is not right here

KeyError: 'data'